In [1]:
import matplotlib as plt 
import numpy as np 
from matplotlib import colors
from PIL import Image, ImageDraw
import random
import pandas as pd
from collections import defaultdict
import random
from collections import deque
from sys import maxsize
import pickle

In [2]:
def createList(r1, r2):#option creation for random edge addition
    if (r1 == r2):
        return r1
    else:
        res = []
        while(r1 != r2):
            res.append(r1)
            r1 += 1
            if (r1 > 50):
                r1 = r1-50
        res.append(r2)
    return res 

In [3]:
class Graph():
    
    def __init__(self):
        
        #create graph
        self.graph = {}
        for i in range(1,51):
            if i-1 == 0:
                self.graph[i] = [2, 50]
            elif i+1 == 51:
                self.graph[i] = [1, 49]
            else:
                self.graph[i] = [i+1, i-1]
        
        #create list of nodes
        self.nodes = createList(1, 50)
        self.paths = self.all_shortest_paths()
        
    def find_paths(self, paths, path, parent, end):
        if end == -1:
            paths.append(path.copy())
            return

        for par in parent[end]:
            path.append(end)
            self.find_paths(paths, path, parent, par)
            path.pop()
        
    def bfs_all_paths(self, parent, start):
    
        dist = [maxsize for _ in range(51)]
        q = deque()
        q.append(start)
        parent[start] = [-1]
        dist[start] = 0

        while q:
            u = q[0]
            q.popleft()
            for v in self.graph[u]:
                if dist[v] > dist[u] + 1:
                    dist[v] = dist[u] + 1
                    q.append(v)
                    parent[v].clear()
                    parent[v].append(u)

                elif dist[v] == dist[u] + 1:
                    parent[v].append(u)

    def all_shortest_paths(self):
        shortest_paths = dict()
        for i in range(1, 51):
            for j in range(1, 51):
                paths, path, shortest_path = [], [], []

                parent = [[] for _ in range(51)]
                self.bfs_all_paths(parent, i)

                self.find_paths(paths, path, parent, j)

                for move in paths:
                    move = list(reversed(move))
                    shortest_path.append(move)
                    
                shortest_paths[i,j] = shortest_path

        return shortest_paths

                
    def addEdge(self):
        boo1 = True
        while boo1:
            if len(self.nodes) == 0:
                boo1 = False
                break
            val = random.choice(self.nodes)
            self.nodes.remove(val)
            valstart = val-5
            valend = val+5
            if (valstart < 1):
                valstart = valstart+50
            if (valend > 50):
                valend = valend-50
            val_nodes = createList(valstart, valend)
            if(val == 1): #deleting adjacent options so that double edges arent made
                val_nodes.remove(50)
            elif(val == 50):
                val_nodes.remove(1)
            else: 
                val_nodes.remove(val-1)
                val_nodes.remove(val+1)
            boo2 = False
            while len(val_nodes) > 0:
                newnode = random.choice(val_nodes)
                if newnode in self.nodes:
                    self.nodes.remove(newnode)
                    self.graph[val].append(newnode)
                    self.graph[newnode].append(val)
                    boo2 = True
                    break
                else:
                    val_nodes.remove(newnode)
                    
            if boo2:
                break
        if boo1 == False:
            return False
        else:
            return True

In [4]:
#start position, end position, backwards path to take
#returns path
def BFS(start, end, graph):
 
    # a queue of the vertices whose path is to be scanned
    # boolean array for checking if node has been visited at least once or not
    # all moves in path set to -1
    queue = []
    visited = [False for i in range(51)]
    back_path = [-1 for i in range(51)]
    
    # start is visited and added to the queue
    visited[start] = True
    queue.append(start)

    # BFS algorithm - look at all neighbors first
    while (len(queue) != 0):
        u = queue[0]
        queue.pop(0)
        for i in range(len(graph[u])):
            if (visited[graph[u][i]] == False):
                visited[graph[u][i]] = True
                back_path[graph[u][i]] = u
                queue.append(graph[u][i])

                # We stop BFS when we find
                # destination.
                if (graph[u][i] == end):
                    path = []
                    crawl = end
                    path.append(crawl)
                    
                    #reversing back_path and removing -1 from nodes that are not in the path to get full path
                    while (back_path[crawl] != -1):
                        path.insert(0, back_path[crawl])
                        crawl = back_path[crawl]
                    return path

    return [end]

In [5]:
class Predator():
    
    def __init__(self, agentloc):
        self.pos = random.randint(1,50)
        #making sure that predator doesn't start where the agent is
        while self.pos == agentloc:
            self.pos = random.randint(1,50)
    
    def pred_distracted_move(self, g, agentpos): #predator moving with probability 0.6 to node closer to agent and probability 0.4 to any other random neighbour
        
        #getting all next possible moves of predator
        neighbors = g.graph[self.pos]
        path = []
        shortest_path = []
        path_length = 50
        #finding shortest path to agent using BFS
        for neighbor in neighbors:
            path = BFS(neighbor, agentpos, g.graph)
            
            #saving shortest path(s)
            if(len(path) <= path_length):
                if(len(path) == path_length):
                    shortest_path.append(path)
                else:
                    shortest_path = []
                    shortest_path.append(path)
                    path_length = len(path)
        
        #randomly picking path of the shortest paths 
        new_path = random.randint(0,(len(shortest_path)-1))
        chance = random.randint(1,100)
        if chance <= 60: #60 percent chance of moving correctly
            self.pos = shortest_path[new_path][0]
        else: #40 percent chance of moving in distracted manner
            self.pos = random.choice(neighbors)
        return self
        

In [6]:
class Prey():
    
    def __init__(self, agentloc):
        self.pos = random.randint(1,50)
        #making sure prey doesn't start where agent is
        while self.pos == agentloc:
            self.pos = random.randint(1,50)
        
    def prey_move(self, g):
        prey_loc_neighbours = [self.pos]
        for i in g.graph[self.pos]:
            prey_loc_neighbours.append(i)
        #randomly choose neighbor to move to (or staying put)
        self.pos = random.choice(prey_loc_neighbours)
        return self

In [7]:
f = open('ustar-dion.p', 'rb')
utility = pickle.load(f)
f.close()
utility

{(1, 1, 1): 1000,
 (1, 1, 2): 0,
 (1, 1, 3): 0,
 (1, 1, 4): 0,
 (1, 1, 5): 0,
 (1, 1, 6): 0,
 (1, 1, 7): 0,
 (1, 1, 8): 0,
 (1, 1, 9): 0,
 (1, 1, 10): 0,
 (1, 1, 11): 0,
 (1, 1, 12): 0,
 (1, 1, 13): 0,
 (1, 1, 14): 0,
 (1, 1, 15): 0,
 (1, 1, 16): 0,
 (1, 1, 17): 0,
 (1, 1, 18): 0,
 (1, 1, 19): 0,
 (1, 1, 20): 0,
 (1, 1, 21): 0,
 (1, 1, 22): 0,
 (1, 1, 23): 0,
 (1, 1, 24): 0,
 (1, 1, 25): 0,
 (1, 1, 26): 0,
 (1, 1, 27): 0,
 (1, 1, 28): 0,
 (1, 1, 29): 0,
 (1, 1, 30): 0,
 (1, 1, 31): 0,
 (1, 1, 32): 0,
 (1, 1, 33): 0,
 (1, 1, 34): 0,
 (1, 1, 35): 0,
 (1, 1, 36): 0,
 (1, 1, 37): 0,
 (1, 1, 38): 0,
 (1, 1, 39): 0,
 (1, 1, 40): 0,
 (1, 1, 41): 0,
 (1, 1, 42): 0,
 (1, 1, 43): 0,
 (1, 1, 44): 0,
 (1, 1, 45): 0,
 (1, 1, 46): 0,
 (1, 1, 47): 0,
 (1, 1, 48): 0,
 (1, 1, 49): 0,
 (1, 1, 50): 0,
 (1, 2, 1): 1000,
 (1, 2, 2): 4.568485978553405,
 (1, 2, 3): 1,
 (1, 2, 4): 1,
 (1, 2, 5): 1,
 (1, 2, 6): 1,
 (1, 2, 7): 1,
 (1, 2, 8): 1,
 (1, 2, 9): 1,
 (1, 2, 10): 1,
 (1, 2, 11): 1,
 (1, 2, 12): 1,
 (1,

In [ ]:
util = []
for pos,val in data.items():
    local = {}
    local["Agent"] = pos[0]
    local["Prey"] = pos[1]
    local["Predator"] = pos[2]
    local["Distance between Agent and Prey"] = len(path[pos[0],pos[1]][0])
    local["Distance between Agent and Predator"] = len(path[pos[0],pos[2]][0])
    local["Distance between Prey and Predator"] = len(path[pos[2],pos[1]][0])
    local["Ustar"] = val
    util.append(local)

total = pd.DataFrame(l)
total_copy = total.copy()
total = total[total['Ustar'] < 100]

X = total.iloc[:,:-1].values
y = total.iloc[:,-1].values
y = total.expand_dims(y,axis = 1)

X_train = []
y_train = []
X_test = []
y_test = []

for i in range(len(y)):
    r = np.random.randn()
    if r < 0.70:
        X_train.append(df.iloc[i,:-1].values)
        y_train.append(df.iloc[i,-1])
    else:
        X_test.append(df.iloc[i,:-1].values)
        y_test.append(df.iloc[i,-1])

In [ ]:
X_train = np.array(X_train)
y_train = np.array(y_train)
y_train = np.expand_dims(y_train, axis = 1)
y_test = np.expand_dims(y_test, axis = 1)
X_test = np.array(X_test)
y_test = np.array(y_test)

In [ ]:
class NN:
    def __init__(self,X,Y):
        self.W1 = np.random.randn(36, 6)*0.01
        self.b1 = np.zeros((36, 1))
        self.W2 = np.random.randn(1, 36)*0.01
        self.b2 = np.zeros((1, 1))
        self.loss_list = []
        self.Z1 = None
        self.Z2 = None
        self.A1 = None
        self.A2 = None
        self.X = X.T
        self.m = self.X.shape[1]
        self.Y = Y.T
    
    def relu(self,A1):
        return np.maximum(A1,0)
    
    def relu_prime(self,Z1):
        return 1. * (Z1 > 0)
    
    def leaky_relu(self,A1):
        A1[A1<0] = A1[A1<0]*0.01
        return A1
    
    def leaky_relu_prime(self,Z1):
        Z1[Z1<0] = 0.01
        Z1[Z1>=0] = 1
        return Z1
    
    def forward(self):
        self.Z1 = np.dot(self.W1,self.X) + self.b1
        self.A1 = self.leaky_relu(self.Z1)
        self.Z2 = np.dot(self.W2,self.A1) + self.b2
        self.A2 = self.Z2
        
    def backward(self):
        dZ2 = (self.A2-self.Y)
        dW2 = (1/self.m)*(np.dot(dZ2,self.A1.T))
        db2 = (1/self.m) * np.sum(dZ2)
        dZ1 = np.multiply(np.dot(self.W2.T,dZ2), self.leaky_relu_prime(self.Z1))
        dW1 = (1/self.m)*(np.dot(dZ1,self.X.T))
        db1 = (1/self.m)*(np.sum(dZ1))
        return dW1, db1, dW2, db2
    
    def update_params(self,dW1, db1, dW2, db2):
        self.W1 = self.W1 - 0.001*dW1
        self.b1 = self.b1 - 0.001*db1
        self.W2 = self.W2 - 0.001*dW2
        self.b2 = self.b2 - 0.001*db2
        
    def run(self):
        for i in range(3000):
            self.forward()
            dW1, db1, dW2, db2 = self.backward()
            self.update_params(dW1, db1, dW2, db2)
            loss = (np.mean((self.Y - self.A2)**2)*0.5)
            self.loss_list.append(loss)
            print("Loss for iteration {} => {}".format(i,loss))
    
    
    def predict(self, X):
        Z1 = np.dot(obj.W1,X) + obj.b1
        A1 = obj.relu(Z1)
        Z2 = np.dot(obj.W2,A1) + obj.b2
        A2 = Z2
        return A2

In [ ]:
obj = NN(X,y)
obj.run()

In [ ]:
U_star= {}
for i in range(125000):
    X = df1.iloc[i,:-1].values
    X = np.expand_dims(X,axis = 1)
    y = predict(X)
    U_star[X[0][0],X[1][0],X[2][0]] = y[0]

In [ ]:
def AgentVStar(g, utility):
    agent_pos = random.randint(1,50)
    predator = Predator(agent_pos)
    prey = Prey(agent_pos)
    fail = 0
    steps = 0
    path = []
    path.append(agent_pos)
    
    while (fail == 0 and steps <= 100):
        
        #move agent to the neighbor with the lowest utility
        best_node = agent_pos
        min_utility = 10000
        
        for action in g.graph[agent_pos]:
            if utility.Ustar[action][prey.pos][predator.pos] < min_utility:
                min_utility = utility.Ustar[action][prey.pos][predator.pos]
                best_node = action
        
        agent_pos = best_node
        path.append(agent_pos)
        steps += 1
        
        if (agent_pos == prey.pos): #if agent has already moved to location of prey
            break

        #move prey
        prey.prey_move(g)

        if (prey.pos == agent_pos): #if prey luckily moves to location of agent
            break

        #move predator
        predator.pred_distracted_move(g, agent_pos)

        if(predator.pos == agent_pos): #if predator catches agent
            fail = 1
            break
        
    #fail if timeout error    
    if(steps >= 100):
        fail = 2 

    return fail, path

In [ ]:
graph = {1: [2, 50, 5], 2: [3, 1, 50], 3: [4, 2, 7], 4: [5, 3, 9], 5: [6, 4, 1], 6: [7, 5, 8], 7: [8, 6, 3], 8: [9, 7, 6], 9: [10, 8, 4], 10: [11, 9, 14], 11: [12, 10], 12: [13, 11, 16], 13: [14, 12, 18], 14: [15, 13, 10], 15: [16, 14, 19], 16: [17, 15, 12], 17: [18, 16, 22], 18: [19, 17, 13], 19: [20, 18, 15], 20: [21, 19, 24], 21: [22, 20, 26], 22: [23, 21, 17], 23: [24, 22, 27], 24: [25, 23, 20], 25: [26, 24, 29], 26: [27, 25, 21], 27: [28, 26, 23], 28: [29, 27, 30], 29: [30, 28, 25], 30: [31, 29, 28], 31: [32, 30, 34], 32: [33, 31], 33: [34, 32, 36], 34: [35, 33, 31], 35: [36, 34, 39], 36: [37, 35, 33], 37: [38, 36, 40], 38: [39, 37, 41], 39: [40, 38, 35], 40: [41, 39, 37], 41: [42, 40, 38], 42: [43, 41, 45], 43: [44, 42, 48], 44: [45, 43, 46], 45: [46, 44, 42], 46: [47, 45, 44], 47: [48, 46, 49], 48: [49, 47, 43], 49: [50, 48, 47], 50: [1, 49, 2]}

track = {"Success":0,"Dead":0,"Timeout":0}
for _ in range(3000):
    status = AgentVStar(graph,U_star)
    track[status] += 1

print(track)


In [ ]:
wins = 0
losses = 0
timeout = 0
times = 0

while times < 3000:
    
    res, path = AgentVStar(g, utility)
    
    if(res == 0):
        wins += 1
    elif(res == 1):
        losses += 1
    else:
        timeout += 1
    times += 1
    
print('Number of wins: ' + str(wins))
print('Number of losses: ' + str(losses))
print('Number of timeouts: ' + str(timeout))
print("Success Rate: " + str((wins/times)))